In [1]:
import pandas as pd
import numpy as np
import math
import datetime

In [2]:
train = pd.read_csv('data/train_users_2.csv')
test = pd.read_csv('data/test_users.csv')
countries = pd.read_csv('data/countries.csv')
user_demo = pd.read_csv('data/age_gender_bkts.csv')
sessions = pd.read_csv('data/sessions.csv')
#changing into datetime
train.date_account_created = pd.to_datetime(train.date_account_created)
train.timestamp_first_active = pd.to_datetime(train.timestamp_first_active, format = "%Y%m%d%H%M%S")
train.date_first_booking = pd.to_datetime(train.date_first_booking)
test.timestamp_first_active = pd.to_datetime(test.timestamp_first_active, format = "%Y%m%d%H%M%S")
test.date_account_created = pd.to_datetime(test.date_account_created)

train_destination = train.iloc[:,-1]

In [3]:
unknowntrain = [i for i, j in enumerate(train.gender) if j == '-unknown-']
train.loc[unknowntrain, 'gender'] = 'NA'
unknowntest = [i for i, j in enumerate(test.gender) if j == '-unknown-']
test.loc[unknowntest, 'gender'] = 'NA'

#unknowntest = test.gender.index('-unknown-')
#test.loc[unknowntest, 'gender'] = 'NA'

In [4]:
np.unique(train.gender)

array(['FEMALE', 'MALE', 'NA', 'OTHER'], dtype=object)

In [5]:
#sessions grouping by user

#Group by user_id, aggregate by number of counts (counting device_type as it is never NA), 
#and total sum of elapsed time in seconds
group_sessions = sessions.groupby("user_id").agg({'device_type':'count', 'secs_elapsed':'sum'})
#rename columns
group_sessions.columns = ['sum_secs_elapsed', 'counts']
#group by variable turns into index, I'm reseting the index and putting user_id back as a column
group_sessions.reset_index(level=0, inplace=True)

In [6]:
group_sessions.head()

,user_id,sum_secs_elapsed,counts
0,00023iyk9l,867896.0,40
1,0010k6l0om,586543.0,63
2,001wyh0pz8,282965.0,90
3,0028jgx1x1,297010.0,31
4,002qnbzfs5,6487080.0,789


In [7]:
#bucket all ages into format that user_demo is in for age
def agebuckets(ages):
    ageless =  [i for i in range(5,101,5)] # 5, 10, 15, 20...95, 100
    buckets = ['%d-%d' %(i, i+4) for i in range(0,100,5)] # 0-4, 5-9, 10-14...90-94, 95-99
    newlist = []
    for i in range(len(ages)):
        if math.isnan(ages[i]):
            newlist.append('NA')
        elif ages[i] <ageless[0]:
            newlist.append(buckets[0])
        elif ages[i] < ageless[1]:
            newlist.append(buckets[1])
        elif ages[i] < ageless[2]:
            newlist.append(buckets[2])
        elif ages[i] < ageless[3]:
            newlist.append(buckets[3])
        elif ages[i] < ageless[4]:
            newlist.append(buckets[4])
        elif ages[i] < ageless[5]:
            newlist.append(buckets[5])
        elif ages[i] < ageless[6]:
            newlist.append(buckets[6])
        elif ages[i] < ageless[7]:
            newlist.append(buckets[7])
        elif ages[i] < ageless[8]:
            newlist.append(buckets[8])
        elif ages[i] < ageless[9]:
            newlist.append(buckets[9])
        elif ages[i] < ageless[10]:
            newlist.append(buckets[10])
        elif ages[i] < ageless[11]:
            newlist.append(buckets[11])
        elif ages[i] < ageless[12]:
            newlist.append(buckets[12]) 
        elif ages[i] < ageless[13]:
            newlist.append(buckets[13]) 
        elif ages[i] < ageless[14]:
            newlist.append(buckets[14])
        elif ages[i] < ageless[15]:
            newlist.append(buckets[15])
        elif ages[i] < ageless[16]:
            newlist.append(buckets[16])
        elif ages[i] < ageless[17]:
            newlist.append(buckets[17])
        elif ages[i] < ageless[18]:
            newlist.append(buckets[18])
        elif ages[i] < ageless[19]:
            newlist.append(buckets[19]) 
        else:
            newlist.append('100+')
    return newlist

In [8]:
train.age[0:20]

0      NaN
1     38.0
2     56.0
3     42.0
4     41.0
5      NaN
6     46.0
7     47.0
8     50.0
9     46.0
10    36.0
11    47.0
12     NaN
13    37.0
14    36.0
15    33.0
16     NaN
17    31.0
18     NaN
19    29.0
Name: age, dtype: float64

In [9]:
train.age = agebuckets(train.age)
test.age = agebuckets(test.age)

In [10]:
def timedif(L1, L2):
    timediflist = []
    for i in range(len(L1)):
        try:
            if (L1[i]-L2[i]).days <= -1:#datetime.timedelta(days=0):
                timediflist.append('before')
            elif (L1[i]-L2[i]).days ==0: #datetime.timedelta(days=1):
                timediflist.append('same day')
            else:
                timediflist.append('greater 1 day')
        except:
            timediflist.append('NB')
            
    return timediflist

In [11]:
#adding time lag columns
train['lag_account_created'] = timedif(train.date_first_booking, train.date_account_created)
train['lag_first_active'] = timedif(train.date_first_booking, train.timestamp_first_active)
train['lag_account_created_first_active'] = timedif(train.date_account_created, train.timestamp_first_active)
test['lag_account_created_first_active'] = timedif(test.date_account_created, test.timestamp_first_active)

In [12]:
def bookings(L1, L2, L3, L4):
    timediflist = []
    for i in range(len(L1)):
        if L1[i] == 'same day' or L2[i] == 'same day':
            timediflist.append('early')
        elif L1[i] == 'before' and L2[i] == 'before' and L3[i] == 'same day':
            timediflist.append('early')
        elif L1[i] == 'greater 1 day' and L2[i] == 'greater 1 day':
            timediflist.append('waited')
        elif L1[i] == 'greater 1 day' and L2[i] == 'before':
            timediflist.append('waited')
        elif L1[i] == 'before' and L2[i] == 'greater 1 day':
            timediflist.append('waited')
        elif L1[i] == 'before' and L2[i] == 'before' and L3[i] == 'greater 1 day':
            timediflist.append('waited')
        elif L4[i] == 'NDF':
            timediflist.append('NB')
        else:
            timediflist.append('NA')

            
    return timediflist

In [13]:
booking = bookings(train.lag_account_created, train.lag_first_active, train.lag_account_created_first_active, train.country_destination)

In [14]:
train['bookings'] = booking

In [15]:
#given the train data gender, age, and country_desination produce the corresponding population in thousands
population_in_thous = []
for i in range(train.shape[0]):
    if train.country_destination[i] == 'NDF':
        population_in_thous.append('NB')    
    elif train.gender[i] == 'NA' or train.age[i] == 'NA' or train.gender[i] == 'nan': 
        population_in_thous.append('NA')
    elif train.gender[i] == 'OTHER':
        population_in_thous.append(0)  
    elif train.country_destination[i] == 'other':
        gendersi = user_demo.loc[user_demo.gender == train.gender[i].lower(),:] 
        ages = gendersi.loc[gendersi.age_bucket == train.age[i], :]
        ages = list(map(lambda x: float(x), ages.population_in_thousands))
        population_in_thous.append(np.mean(ages))
    else:
        genders = user_demo.loc[user_demo.gender == train.gender[i].lower(),:] 
        dests = genders.loc[genders.country_destination == train.country_destination[i] ,:]    
        population_in_thous.append(float((dests.loc[dests.age_bucket == train.age[i], 'population_in_thousands'])))
        
population_in_thous[0:10]

['NB',
 'NB',
 11264.0,
 2458.8000000000002,
 'NA',
 'NA',
 10659.0,
 10659.0,
 11413.0,
 'NA']

In [16]:
#merging gender age bucket with train data
train['population_in_thousands'] = population_in_thous

In [17]:
#merging with grouped sessions and countries, **note most of training data is not in sessions. see below 
test_m = pd.merge(test, group_sessions, left_on='id', right_on ='user_id', how='left')
train_m = pd.merge(train, group_sessions, left_on='id', right_on ='user_id', how='left')
test_m = test_m.drop('user_id', 1)
train_m = train_m.drop('user_id', 1)
print train_m.iloc[0:5, 0:10] #to be deleted?
print train_m.iloc[0:5, 10:]  # to be deleted?

           id date_account_created timestamp_first_active date_first_booking  \
0  gxn3p5htnn           2010-06-28    2009-03-19 04:32:55                NaT   
1  820tgsjxq7           2011-05-25    2009-05-23 17:48:09                NaT   
2  4ft3gnwmtx           2010-09-28    2009-06-09 23:12:47         2010-08-02   
3  bjjt8pjhuk           2011-12-05    2009-10-31 06:01:29         2012-09-08   
4  87mebub9p4           2010-09-14    2009-12-08 06:11:05         2010-02-18   

   gender    age signup_method  signup_flow language affiliate_channel  
0      NA     NA      facebook            0       en            direct  
1    MALE  35-39      facebook            0       en               seo  
2  FEMALE  55-59         basic            3       en            direct  
3  FEMALE  40-44      facebook            0       en            direct  
4      NA  40-44         basic            0       en            direct  
  affiliate_provider first_affiliate_tracked signup_app first_device_type  \
0   

In [18]:
train_m.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,...,first_device_type,first_browser,country_destination,lag_account_created,lag_first_active,lag_account_created_first_active,bookings,population_in_thousands,sum_secs_elapsed,counts
0,gxn3p5htnn,2010-06-28,2009-03-19 04:32:55,NaT,NA,NA,facebook,0,en,direct,...,Mac Desktop,Chrome,NDF,NB,NB,greater 1 day,NB,NB,NaN,NaN
1,820tgsjxq7,2011-05-25,2009-05-23 17:48:09,NaT,MALE,35-39,facebook,0,en,seo,...,Mac Desktop,Chrome,NDF,NB,NB,greater 1 day,NB,NB,NaN,NaN
2,4ft3gnwmtx,2010-09-28,2009-06-09 23:12:47,2010-08-02,FEMALE,55-59,basic,3,en,direct,...,Windows Desktop,IE,US,before,greater 1 day,greater 1 day,waited,11264,NaN,NaN
3,bjjt8pjhuk,2011-12-05,2009-10-31 06:01:29,2012-09-08,FEMALE,40-44,facebook,0,en,direct,...,Mac Desktop,Firefox,other,greater 1 day,greater 1 day,greater 1 day,waited,2458.8,NaN,NaN
4,87mebub9p4,2010-09-14,2009-12-08 06:11:05,2010-02-18,NA,40-44,basic,0,en,direct,...,Mac Desktop,Chrome,US,before,greater 1 day,greater 1 day,waited,NA,NaN,NaN


In [19]:
train_m.columns

Index([u'id', u'date_account_created', u'timestamp_first_active',
       u'date_first_booking', u'gender', u'age', u'signup_method',
       u'signup_flow', u'language', u'affiliate_channel',
       u'affiliate_provider', u'first_affiliate_tracked', u'signup_app',
       u'first_device_type', u'first_browser', u'country_destination',
       u'lag_account_created', u'lag_first_active',
       u'lag_account_created_first_active', u'bookings',
       u'population_in_thousands', u'sum_secs_elapsed', u'counts'],
      dtype='object')

In [20]:
train_m.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,...,first_device_type,first_browser,country_destination,lag_account_created,lag_first_active,lag_account_created_first_active,bookings,population_in_thousands,sum_secs_elapsed,counts
0,gxn3p5htnn,2010-06-28,2009-03-19 04:32:55,NaT,NA,NA,facebook,0,en,direct,...,Mac Desktop,Chrome,NDF,NB,NB,greater 1 day,NB,NB,NaN,NaN
1,820tgsjxq7,2011-05-25,2009-05-23 17:48:09,NaT,MALE,35-39,facebook,0,en,seo,...,Mac Desktop,Chrome,NDF,NB,NB,greater 1 day,NB,NB,NaN,NaN
2,4ft3gnwmtx,2010-09-28,2009-06-09 23:12:47,2010-08-02,FEMALE,55-59,basic,3,en,direct,...,Windows Desktop,IE,US,before,greater 1 day,greater 1 day,waited,11264,NaN,NaN
3,bjjt8pjhuk,2011-12-05,2009-10-31 06:01:29,2012-09-08,FEMALE,40-44,facebook,0,en,direct,...,Mac Desktop,Firefox,other,greater 1 day,greater 1 day,greater 1 day,waited,2458.8,NaN,NaN
4,87mebub9p4,2010-09-14,2009-12-08 06:11:05,2010-02-18,NA,40-44,basic,0,en,direct,...,Mac Desktop,Chrome,US,before,greater 1 day,greater 1 day,waited,NA,NaN,NaN


In [506]:
#remove id, time format columns and categorical columns
excl = list(train_m.columns)
toremove = ['id', 'date_account_created', 'timestamp_first_active', 'date_first_booking', 'population_in_thousands',
            'lag_account_created', 'bookings', 'lag_first_active', 
            #'lag_account_created_first_active', 
              'country_destination'] 
map(lambda x: excl.remove(x), toremove)


[None, None, None, None, None, None, None, None, None]

In [507]:
train_m.loc[0:10, excl]

,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,lag_account_created_first_active,sum_secs_elapsed,counts
0,NA,NA,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,greater 1 day,NaN,NaN
1,MALE,35-39,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,greater 1 day,NaN,NaN
2,FEMALE,55-59,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,greater 1 day,NaN,NaN
3,FEMALE,40-44,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,greater 1 day,NaN,NaN
4,NA,40-44,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,greater 1 day,NaN,NaN
5,NA,NA,basic,0,en,other,other,omg,Web,Mac Desktop,Chrome,before,NaN,NaN
6,FEMALE,45-49,basic,0,en,other,craigslist,untracked,Web,Mac Desktop,Safari,before,NaN,NaN
7,FEMALE,45-49,basic,0,en,direct,direct,omg,Web,Mac Desktop,Safari,before,NaN,NaN
8,FEMALE,50-54,basic,0,en,other,craigslist,untracked,Web,Mac Desktop,Safari,before,NaN,NaN
9,NA,45-49,basic,0,en,other,craigslist,omg,Web,Mac Desktop,Firefox,before,NaN,NaN


In [508]:
train_xcl = train_m.loc[:, excl]
test_xcl = test_m.loc[:, excl]

In [509]:
print train_xcl.iloc[0,:]

gender                                         NA
age                                            NA
signup_method                            facebook
signup_flow                                     0
language                                       en
affiliate_channel                          direct
affiliate_provider                         direct
first_affiliate_tracked                 untracked
signup_app                                    Web
first_device_type                     Mac Desktop
first_browser                              Chrome
lag_account_created_first_active    greater 1 day
sum_secs_elapsed                              NaN
counts                                        NaN
Name: 0, dtype: object


In [510]:
#train_xcl.to_csv("train_x_pre_dic623nopop.csv")
#test_xcl.to_csv("test_x_pre_dic623nopop.csv")

In [511]:
from sklearn.feature_extraction import DictVectorizer

frames = [train_xcl, test_xcl]
all_data = pd.concat(frames)
all_data = all_data.T.to_dict().values()
train_xdic = train_xcl.T.to_dict().values()
test_xdic = test_xcl.T.to_dict().values()

vec = DictVectorizer()
vec.fit(all_data)
train_xvec = vec.transform(train_xdic)
test_xvec = vec.transform(test_xdic)

In [512]:
train_y = train_m.loc[:,'country_destination']

In [513]:
#train_y = train_y.T.to_dict().values()
#vec1 = DictVectorizer()
#vec1.fit(train_y)
#train_y = vec.transform(train_y)

In [514]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(train_y)
train_ytrans = le.transform(train_y)
print type(train_ytrans)

<type 'numpy.ndarray'>


In [515]:
#from sklearn import preprocessing
#oh = preprocessing.OneHotEncoder()
#train_y = train_y.to_dict().values()
#print train_y
#enc.transform([[0, 1, 1]]).toarray()
#oh.fit(train_y)
#train_y = train_y.T.to_dict().values()
#print train_y
#train_y = oh.fit_transform(train_y)
#train_y = oh.transform(train_y).toarray()
#print type(train_y)

In [516]:
set(train_ytrans)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}

In [517]:
print train_xcl.columns

Index([u'gender', u'age', u'signup_method', u'signup_flow', u'language',
       u'affiliate_channel', u'affiliate_provider', u'first_affiliate_tracked',
       u'signup_app', u'first_device_type', u'first_browser',
       u'lag_account_created_first_active', u'sum_secs_elapsed', u'counts'],
      dtype='object')


In [538]:
#model was fit using xgboost
import xgboost as xgb
import numpy as np
params = {}
params["objective"] = "multi:softmax"
params["num_class"] = 12
params["eta"] = 0.3
params["min_child_weight"] = 6
params["subsample"] = 0.7
params["colsample_bytree"] = 0.7
params["scale_pos_weight"] = 1
params["max_depth"] = 3
params['eval_metric'] = "ndcg@5"
params['nthread'] = 4


plst = list(params.items())

dtrain = xgb.DMatrix(train_xvec, label=train_ytrans)
dtest = xgb.DMatrix(test_xvec)
num_round = 25
model = xgb.train(plst, dtrain, num_round, verbose_eval=True)


In [571]:
#This model2 was fit using xgboost.XGBClassifier
import numpy as np

model2 = xgb.XGBClassifier(max_depth = 5, learning_rate = 0.1, n_estimators = 30, silent = False, \
                           objective ="multi:softprob")


In [572]:
model2.fit(train_xvec, train_ytrans, verbose = True, eval_metric = "ndcg@5")

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=30, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=False, subsample=1)

In [583]:
pred2 = model2.predict(train_xvec)
pred2

array([ 7,  7, 10, ..., 10,  7,  7])

In [642]:
pred = map(int,model.predict(dtrain))
print pred[0]

7


In [566]:
print test_x.shape
print train_x.shape

(62096, 14)
(213451, 14)


In [567]:
pred2inv = le.inverse_transform(pred2)

In [541]:
predinv = le.inverse_transform(pred)

In [568]:
import sklearn.metrics as me
me.accuracy_score(pred2inv, train_y)

0.63115188029102698

In [543]:
import sklearn.metrics as me
me.accuracy_score(predinv, train_y)

0.6331429695808406

In [525]:
#testing the first model with the test data to be submitted to kaggle
pred1 = model.predict(dtest)
pred1 = map(int,pred1)
pred1 = le.inverse_transform(pred1)
print set(pred1)

set(['US', 'NDF'])


In [529]:
#turning the test results into kaggle format
testsub = pd.DataFrame({'id': test_m.id, 'country':pred1})
testsub = testsub[['id', 'country']]
print testsub.iloc[0:3,:]

           id country
0  5uwns89zht     NDF
1  jtl0dijy2j     NDF
2  xx0ulgorjt     NDF


In [530]:
#testsub.to_csv("testsub1yk.csv", index=False)

In [469]:
#confusing matrix on the first model
print me.confusion_matrix(predinv, train_y)

[[     0      0      0      0      0      0      0      0      0      0
       0      0]
 [     0      0      0      0      0      0      0      0      0      0
       0      0]
 [     0      0      0      0      0      0      0      0      0      0
       0      0]
 [     0      0      0      0      0      0      0      0      0      0
       0      0]
 [     0      0      0      0      0      0      0      0      0      0
       0      0]
 [     0      0      0      0      0      0      0      0      0      0
       0      0]
 [     0      0      0      0      0      0      0      0      0      0
       0      0]
 [   246    667    533   1175   2547   1139   1471 102214    359    122
   30354   5145]
 [     0      0      0      0      0      0      0      0      0      0
       0      0]
 [     0      0      0      0      0      0      0      0      0      0
       0      0]
 [   293    761    528   1074   2476   1185   1364  22329    403     95
   32022   4949]
 [     0      0      

In [201]:
import sklearn.metrics as me
me.accuracy_score(pred, le.inverse_transform(train_y))

0.93311345461019157

In [70]:
print train_m.columns
print test_m.columns

Index([u'id', u'date_account_created', u'timestamp_first_active',
       u'date_first_booking', u'gender', u'age', u'signup_method',
       u'signup_flow', u'language', u'affiliate_channel',
       u'affiliate_provider', u'first_affiliate_tracked', u'signup_app',
       u'first_device_type', u'first_browser', u'country_destination',
       u'lag_account_created', u'lag_first_active',
       u'lag_account_created_first_active', u'bookings',
       u'population_in_thousands', u'sum_secs_elapsed', u'counts'],
      dtype='object')
Index([u'id', u'date_account_created', u'timestamp_first_active',
       u'date_first_booking', u'gender', u'age', u'signup_method',
       u'signup_flow', u'language', u'affiliate_channel',
       u'affiliate_provider', u'first_affiliate_tracked', u'signup_app',
       u'first_device_type', u'first_browser',
       u'lag_account_created_first_active', u'sum_secs_elapsed', u'counts'],
      dtype='object')


In [576]:
from sklearn import tree
tree_model = tree.DecisionTreeClassifier()


In [623]:
#dictionary to get rid of the f in the feature number when calling model.get_fscore()
import re
def ranked (dic):
    redic = {}
    for b,d in dic.items():
        redic[int(re.findall('f(\d*)',b)[0])] = d
    return redic

In [637]:
#getting rid of f in feature number when calling model.f_score od is f_number and associated importance
franking = ranked(model.get_fscore())
import collections
od = collections.OrderedDict(sorted(franking.items()))
print od

print od[1]

OrderedDict([(0, 4), (1, 27), (2, 10), (3, 59), (4, 5), (5, 21), (6, 13), (7, 8), (9, 3), (10, 12), (12, 8), (14, 8), (15, 3), (16, 15), (18, 5), (20, 9), (22, 1), (26, 1), (27, 7), (28, 41), (29, 18), (30, 38), (31, 28), (32, 2), (33, 2), (34, 6), (35, 10), (36, 10), (37, 4), (38, 2), (39, 3), (43, 1), (44, 142), (45, 237), (46, 93), (47, 18), (50, 25), (51, 18), (52, 9), (53, 38), (54, 26), (56, 1), (61, 25), (69, 21), (73, 9), (80, 4), (90, 16), (102, 6), (103, 2), (104, 7), (105, 70), (106, 18), (108, 20), (109, 24), (110, 10), (111, 36), (112, 39), (113, 87), (116, 1), (122, 22), (124, 23), (125, 7), (127, 9), (132, 9), (134, 2), (143, 6), (144, 4), (145, 10), (146, 72), (147, 5), (148, 125), (149, 96), (150, 54), (153, 75)])
27


In [638]:
#print out the features and feature importance
listfeatures = list(vec.feature_names_)
for i in od:
    print listfeatures[i], od[i]

affiliate_channel=api 4
affiliate_channel=content 27
affiliate_channel=direct 10
affiliate_channel=other 59
affiliate_channel=remarketing 5
affiliate_channel=sem-brand 21
affiliate_channel=sem-non-brand 13
affiliate_channel=seo 8
affiliate_provider=bing 3
affiliate_provider=craigslist 12
affiliate_provider=direct 8
affiliate_provider=facebook 8
affiliate_provider=facebook-open-graph 3
affiliate_provider=google 15
affiliate_provider=meetup 5
affiliate_provider=other 9
affiliate_provider=vast 1
age=100+ 1
age=15-19 7
age=20-24 41
age=25-29 18
age=30-34 38
age=35-39 28
age=40-44 2
age=45-49 2
age=50-54 6
age=55-59 10
age=60-64 10
age=65-69 4
age=70-74 2
age=75-79 3
age=95-99 1
age=NA 142
counts 237
first_affiliate_tracked 93
first_affiliate_tracked=linked 18
first_affiliate_tracked=omg 25
first_affiliate_tracked=product 18
first_affiliate_tracked=tracked-other 9
first_affiliate_tracked=untracked 38
first_browser=-unknown- 26
first_browser=Android Browser 1
first_browser=Chrome 25
first_br